In [184]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import requests
from bs4 import BeautifulSoup

In [260]:
filename = "web_scraped_pokemon.csv"
df = pd.read_csv(filename).drop(columns= ['Unnamed: 0'])
df[0:10]

,hp,attack,defense,sp_Attack,sp_Defense,speed,total,name,type1,type2
0,70,110,80,95,60,70,485,Flapple,Grass,Dragon
1,70,85,145,60,55,65,480,Orthworm,Steel,
2,38,47,35,33,35,57,245,Rookidee,Flying,
3,40,61,54,45,45,65,310,Sprigatito,Grass,
4,45,45,30,55,40,50,265,Impidimp,Dark,Fairy
5,60,65,65,134,114,70,508,Polteageist,Ghost,
6,125,100,80,85,75,55,520,Dudunsparce,Normal,
7,50,50,45,40,45,75,305,Tandemaus,Normal,
8,50,64,50,38,38,44,284,Chewtle,Water,
9,50,45,45,35,64,58,297,Tinkatink,Fairy,Steel


In [352]:
def type_advantage(attacker,defender):
       
    att_types = []
    att_types.append(attacker['type1'].values[0])
    if attacker['type2'].values[0] != " ":
        att_types.append(attacker['type2'].values[0])
    
    def_types = []
    def_types.append(defender['type1'].values[0])
    if defender['type2'].values[0] != " ":
        def_types.append(defender['type2'].values[0])
    
    
    Effectiveness = []
    
    for type_ in att_types:
        
        super_effective = []
        not_very_effective = []
        
        url = f'https://pokemondb.net/type/{type_}'
        data  = requests.get(url).text
        soup = BeautifulSoup(data,"lxml")
        bm = soup.body.main
        bmd = bm.find('div', class_="grid-col span-md-6 span-lg-12")
        bmdp = bmd.find_all('p', class_="type-fx-list")
        
        
        for a in bmdp[0].find_all('a'):
            super_effective.append(a.text)
            
        for a in bmdp[1].find_all('a'):
            not_very_effective.append(a.text)
        
        
        se = set(super_effective)-set(def_types)
        nve = set(not_very_effective)-set(def_types)
        
        if (len(not_very_effective) - len(nve)) == 1:
            Effectiveness.append(.5)
        if (len(not_very_effective) - len(nve)) == 2:
            Effectiveness.append(.25)
        if (len(not_very_effective) - len(nve)) == 0:
            Effectiveness.append(1)
        if (len(super_effective) - len(se)) == 1:
            Effectiveness.append(2)
        if (len(super_effective) - len(se)) == 2:
            Effectiveness.append(4)

    return  max(Effectiveness)

In [353]:
type_advantage(pokemon1,pokemon2)

2

In [344]:
L = 50

def damage(attacker,defender):
    L=50
    BP = 50
    A = int(attacker['attack'])
    D = int(defender['defense'])
    Sp_A = int(attacker['sp_Attack'])
    Sp_D = int(defender['sp_Defense'])
    
    A_dmg = ((2*L + 10)/250 * A/D * BP + 2) * type_advantage(attacker, defender)
    Sp_A_dmg = ((2*L + 10)/250 * Sp_A/Sp_D * BP + 2) * type_advantage(attacker, defender)
    
    if A_dmg > Sp_A_dmg:
        dmg = A_dmg
        if dmg > ((2*L + 10)/250 * A/D * BP + 2):
            print('      Its super effective!')
        if dmg < ((2*L + 10)/250 * A/D * BP + 2):
            print('      Its not very effective')
    else:
        dmg = Sp_A_dmg
        if dmg > ((2*L + 10)/250 * Sp_A/Sp_D * BP + 2):
            print('      Its super effective!')
        if dmg < ((2*L + 10)/250 * Sp_A/Sp_D * BP + 2):
            print('      Its not very effective')
        
    return int(dmg)

In [338]:
def battle(poke1, poke2):
    poke1_name = poke1['name'].values[0]
    poke2_name = poke2['name'].values[0]
    poke1_hp = poke1['hp'].values[0]
    poke2_hp = poke2['hp'].values[0]
    
    print(f'{poke1_name} has {poke1_hp} starting health')
    print(f'{poke2_name} has {poke2_hp} starting health \n')
    
    victor = ""
    while int(poke1_hp) >0 and int(poke2_hp) >0:
    
        if int(poke1['speed']) > int(poke2['speed']):
            print(f'Your {poke1_name} attacked {poke2_name},')
            dmg = damage(pokemon1, pokemon2)
            poke2_hp = poke2_hp - dmg
            print(f'      {poke2_name} took {dmg} damage, HP: {poke2_hp} \n')
            
            
            if poke2_hp <= 0:
                print(f' \n{poke2_name} fainted \n')
                victor = poke1_name
                break
            else:
                print(f'Their {poke2_name} attacked {poke1_name},')
                dmg = damage(pokemon2, pokemon1)
                poke1_hp = poke1_hp - dmg
                print(f'      {poke1_name} took {dmg} damage, HP: {poke1_hp} \n')
                
        else:
            print(f'Their {poke2_name} attacked {poke1_name},')
            dmg = damage(pokemon2, pokemon1)
            poke1_hp = poke1_hp - dmg
            print(f'      {poke1_name} took {dmg} damage, HP: {poke1_hp} \n')
            
            
            if poke1_hp <= 0:
                print(f' \n{poke1_name} fainted \n')
                victor= poke2_name
                break
                
            else:
                print(f'Your {poke1_name} attacked {poke2_name},')
                dmg = damage(pokemon1, pokemon2)
                poke2_hp = poke2_hp - dmg
                print(f'      {poke2_name} took {dmg} damage, HP: {poke2_hp} \n')
                
        if poke1_hp <= 0:
            print(f'{poke1_name} fainted \n')
            victor = poke2_name
        if poke2_hp <= 0:
            print(f'{poke2_name} fainted \n')
            victor = poke1_name
    print("-------------------------------------------")  
    return victor

In [347]:
pokemon1_name = 'Toedscruel'
pokemon2_name = 'Dudunsparce'

pokemon1 = df[df['name'] == f'{pokemon1_name}']
pokemon2 = df[df['name'] == f'{pokemon2_name}']
pokemon1

,hp,attack,defense,sp_Attack,sp_Defense,speed,total,name,type1,type2
12,80,70,65,80,120,100,515,Toedscruel,Ground,Grass


In [348]:
battle(pokemon1,pokemon2)

Toedscruel has 80 starting health
Dudunsparce has 125 starting health 

Your Toedscruel attacked Dudunsparce,
      Dudunsparce took 25 damage, HP: 100 

Their Dudunsparce attacked Toedscruel,
      Toedscruel took 35 damage, HP: 45 

Your Toedscruel attacked Dudunsparce,
      Dudunsparce took 25 damage, HP: 75 

Their Dudunsparce attacked Toedscruel,
      Toedscruel took 35 damage, HP: 10 

Your Toedscruel attacked Dudunsparce,
      Dudunsparce took 25 damage, HP: 50 

Their Dudunsparce attacked Toedscruel,
      Toedscruel took 35 damage, HP: -25 

Toedscruel fainted 

-------------------------------------------


'Dudunsparce'

In [354]:
total_battles = 10 #len(df)
all_pokemon = df['name'][0:total_battles]
win_counter = 0


for pokemon in all_pokemon:
    pokemon2 = df[df['name'] == f'{pokemon}']
    
    if battle(pokemon1,pokemon2) == pokemon1_name:
        win_counter += 1
win_perc = str((win_counter/total_battles)*100)
print(f'{pokemon1_name} won {win_perc}% of battles')

Toedscruel has 80 starting health
Flapple has 70 starting health 

Your Toedscruel attacked Flapple,
      Its not very effective
      Flapple took 15 damage, HP: 55 

Their Flapple attacked Toedscruel,
      Its super effective!
      Toedscruel took 78 damage, HP: 2 

Your Toedscruel attacked Flapple,
      Its not very effective
      Flapple took 15 damage, HP: 40 

Their Flapple attacked Toedscruel,
      Its super effective!
      Toedscruel took 78 damage, HP: -76 

Toedscruel fainted 

-------------------------------------------
Toedscruel has 80 starting health
Orthworm has 70 starting health 

Your Toedscruel attacked Orthworm,
      Its super effective!
      Orthworm took 68 damage, HP: 2 

Their Orthworm attacked Toedscruel,
      Toedscruel took 30 damage, HP: 50 

Your Toedscruel attacked Orthworm,
      Its super effective!
      Orthworm took 68 damage, HP: -66 

 
Orthworm fainted 

-------------------------------------------
Toedscruel has 80 starting health
Rookide